In [2]:
import os
from os.path import join
import pandas as pd
import numpy as np

# 预设值参数
label_col = 'survival_months'
n_bins = 4
eps = 1e-6
# 读取数据
df_hmu = pd.read_csv('HMU匹配数据生存.csv')



In [3]:
df_hmu = df_hmu.drop_duplicates(['case_id']).copy()

In [8]:
df_hmu

,case_id,gender,age_at_index,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_m,ajcc_pathologic_stage,censor,status,survival_months
0,1400865,1,46,T2,N0,M0,ⅠB,1,0,84.47
1,1400793,1,32,T2,N3a,M0,ⅢA,1,0,105.23
2,1401410,1,61,T3,N2,M0,ⅢA,1,0,66.73
3,1401025,0,57,T4b,N0,M0,ⅢA,1,0,76.67
4,1400716,1,53,T2,N0,M0,ⅠB,1,0,64.10
...,...,...,...,...,...,...,...,...,...,...
2894,1737196,0,59,T2,N0,M0,ⅠB,1,0,35.67
2895,1736252,0,77,T1b,N0,M0,ⅠA,1,0,60.00
2896,1807581,0,67,T1b,N0,M0,ⅠA,1,0,54.73
2897,1814675,1,63,T2,N0,M0,ⅠB,1,0,53.30


In [5]:
hmu_slide_ids = os.listdir('/mnt/usb5/jijianxin/new_wsi/HMU_ALL_PT/')

In [10]:
from collections import defaultdict
case_to_slide_dict = defaultdict(list)
for slide_id in hmu_slide_ids:
    case_id = slide_id.split('.')[0]
    case_to_slide_dict[int(case_id.split('-')[0])].append(slide_id)

In [13]:
df_hmu['slide_id'] = df_hmu['case_id'].map(case_to_slide_dict)

In [16]:
df_tcga = pd.read_csv('tcga匹配数据生存.csv')
tcga_slide_ids1 = os.listdir('/mnt/usb5/jijianxin/new_wsi/20x_tcga_512_feature/pt_files/')
tcga_slide_ids2 = os.listdir('/mnt/usb5/jijianxin/new_wsi/40x_tcga_feature/pt_files/')

In [18]:
tcga_case_to_slide_dict = defaultdict(list)
for slide_id in tcga_slide_ids:
    case_id = '-'.join(slide_id.split('-')[0:3])
    tcga_case_to_slide_dict[case_id].append(slide_id)

In [21]:
df_tcga['slide_id'] = df_tcga['case_id'].map(tcga_case_to_slide_dict)

In [24]:
tcga_df = df_tcga[df_tcga['slide_id'].apply(lambda x: len(x) > 0)]

In [31]:
def add_bins(slide_data):
    assert 'case_id' in slide_data.columns and 'censor' in slide_data.columns
    
    patients_df = slide_data.drop_duplicates(['case_id']).copy()
    uncensored_df = patients_df[patients_df['censor'] < 1]
    disc_labels, q_bins = pd.qcut(uncensored_df[label_col], q=n_bins, retbins=True, labels=False)
    q_bins[-1] = slide_data[label_col].max() + eps
    q_bins[0] = slide_data[label_col].min() - eps

    disc_labels, q_bins = pd.cut(patients_df[label_col], bins=q_bins, retbins=True, labels=False, right=False, include_lowest=True)
    patients_df.insert(2, 'label', disc_labels.values.astype(int))

    patient_dict = {}
    slide_data = slide_data.set_index('case_id')
    for patient in patients_df['case_id']:
        slide_ids = slide_data.loc[patient, 'slide_id']
        if isinstance(slide_ids, str):
            slide_ids = np.array([slide_ids])  # 改为列表再转换为数组
        else:
            slide_ids = np.array(slide_ids)  # 确保 slide_ids 是 numpy 数组
        patient_dict[patient] = slide_ids.tolist()  # 确保字典中的值是列表
        
    return q_bins, patient_dict, patients_df

q_bins, patients_dict, df_hmu_label = add_bins(df_hmu)

# 显示结果
print(q_bins)
print(patients_dict)
print(df_hmu_label)


[9.99990000e-02 9.78500000e+00 1.82700000e+01 3.08500000e+01
 1.09030001e+02]
{1400865: ['1400865.pt'], 1400793: ['1400793.pt'], 1401410: ['1401410.pt'], 1401025: ['1401025.pt'], 1400716: ['1400716.pt'], 1401233: ['1401233.pt'], 1400894: ['1400894.pt'], 1401589: ['1401589.pt'], 1401110: ['1401110.pt'], 1402081: ['1402081.pt'], 1401654: ['1401654.pt'], 1401419: ['1401419.pt'], 1401737: ['1401737.pt'], 1401463: ['1401463.pt'], 1401396: ['1401396.pt'], 1401504: ['1401504.pt'], 1402174: ['1402174.pt'], 1402133: ['1402133.pt'], 1401796: ['1401796.pt'], 1401933: ['1401933.pt'], 1401970: ['1401970.pt'], 1402025: ['1402025.pt'], 1402031: ['1402031.pt'], 1402079: ['1402079.pt'], 1402023: ['1402023.pt'], 1401949: ['1401949.pt'], 1402318: ['1402318.pt'], 1402337: ['1402337.pt'], 1402524: ['1402524.pt'], 1402604: ['1402604.pt'], 1402694: ['1402694.pt'], 1403174: ['1403174.pt'], 1403939: ['1403939.pt'], 1403088: ['1403088.pt'], 1403395: ['1403395.pt'], 1403376: ['1403376.pt'], 1403404: ['1403404.pt

In [34]:
def label_external_data(external_data, q_bins):
    # 确保外部数据包含生存月份列
    assert label_col in external_data.columns
    
    # 使用训练数据的分箱边界对外部数据进行标签化
    disc_labels = pd.cut(external_data[label_col], bins=q_bins, labels=False, right=False, include_lowest=True)
    external_data['label'] = disc_labels.values.astype(int)
    
    return external_data

# 假设 external_df 是你的外部数据 DataFrame
external_df = label_external_data(tcga_df, q_bins)

# 显示结果
print(external_df)


          case_id  status  survival_months  gender age_at_index  \
0    TCGA-3M-AB46       0        58.026761    male           70   
1    TCGA-B7-5816       0        26.695598  female           51   
8    TCGA-BR-6452       0        34.684551  female           78   
9    TCGA-BR-6453       0        15.945031    male           54   
10   TCGA-BR-6455       1        13.873821    male           59   
..            ...     ...              ...     ...          ...   
378  TCGA-VQ-AA6G       1        26.038071    male           68   
379  TCGA-VQ-AA6I       1        16.142289    male           68   
380  TCGA-VQ-AA6J       0        27.550383    male           75   
381  TCGA-VQ-AA6K       1        12.427261    male           59   
382  TCGA-ZA-A8F6       0        17.260085    male           71   

    ajcc_pathologic_t ajcc_pathologic_n ajcc_pathologic_m  \
0                 T2b                N0                MX   
1                 T4a                N0                M0   
8           

/home/stat-jijianxin/anaconda3/envs/clam/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [45]:
del external_df['index']

In [46]:
df_hmu_label.to_csv('HMU_ALL_PT_LABEL.csv',index=None)
external_df.to_csv('TCGA_ALL_PT_LABEL.csv',index=None)